In [61]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from pythainlp.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from sklearn.model_selection import train_test_split
import numpy as np
import pandas
import os
import json
import numpy
import json
import pandas

In [62]:
def LoadJsonFile(filename):
    try:
        with open(filename) as json_data:
            json_file = json.load(json_data)
    except (UnboundLocalError, FileNotFoundError):
        print("No file name", filename, ".json")
        pass
    return json_file

remove_words = LoadJsonFile("MustRemoveWord.json")
print(remove_words)

['à¸„à¸£à¸±à¸š', 'à¸„à¸°', 'à¸„à¹ˆà¸°', ' ', '.']


In [63]:
# Load Datasets
dataframe = pandas.read_csv("Datasets_G_NewLabel.csv", header=None)
dataset = dataframe.values

# Seperate Dataset into input and output datasets
input_dataset = []
output_dataset = []
for data in dataset: # remove frequence words
    sentence = word_tokenize(data[0], engine='deepcut')
    for word in sentence:
        if word in remove_words:
            sentence.remove(word)
    input_dataset.append(sentence)
    output_dataset.append(data[1])
print(input_dataset)

flat_input_list = [item for sublist in input_dataset for item in sublist] # Convert list of list to list
flat_set_of_input_list = list(set(flat_input_list))
flat_set_of_output = list(set(output_dataset))
print("Flat set of Input Datasets",flat_set_of_input_list)
print("Output dataset", flat_set_of_output)

# Encode All of input datasets words from string to number
encoder_input = LabelEncoder()
encoder_input.fit(flat_set_of_input_list)
np.save("Saved_Model/Encoded_Input_classes.npy" , encoder_input.classes_) # Save Encoded Model
print("Encoder Input Classes :",encoder_input.classes_)

# Encode All of output datasets words from string to number
encoder_output = LabelEncoder()
encoder_output.fit(flat_set_of_output)
np.save("Saved_Model/Encoded_Output_classes.npy" , encoder_output.classes_) # Save Encoded Model

number_of_category = len(encoder_output.classes_)
print("Encoder Output has", number_of_category, "Classes :",encoder_output.classes_)

# transform input data to encoded
max_word_lenght = 30
encoded_train_x_datasets = sequence.pad_sequences( [list(encoder_input.transform(sentence)) for sentence in input_dataset], maxlen=max_word_lenght )
print("Encoded Train_X dataset has",len(encoded_train_x_datasets), ":\n", encoded_train_x_datasets)

# transform output data to encoded
encoded_train_y_datasets = np_utils.to_categorical( encoder_output.transform(output_dataset) )
print("Encoded Train_Y dataset :\n", encoded_train_y_datasets)

[['ไม่', 'ทราบ', 'ว่า', 'จะ', 'ติดต่อ', 'อาจารย์', 'สยาม', 'ได้', 'ที่', 'ไหน'], ['ติดต่อ', 'อาจารย์', 'แพรว', 'ได้', 'อย่าง', 'ไร'], ['ติดต่อ', 'อาจารย์', 'อ้อม', 'ได้', 'ยัง', 'ไง'], ['ติดต่อ', 'ดอกเตอร์', 'ได้', 'ยัง', 'ไง'], ['ติดต่อ', 'ดร', 'ได้', 'ยัง', 'ไง'], ['จะ', 'ติดต่อ', 'ดรต้น', 'ได้', 'ที่', 'ไหน'], ['จะ', 'ติดต่อ', 'ดอกเตอร์', 'ชิด', 'ได้', 'ที่', 'ไหน'], ['จะ', 'ติดต่อ', 'อาจารย์', 'ได้', 'ยัง', 'ไง'], ['จะ', 'ติดต่อ', 'อาจารย์', 'อย่าง', 'ไร'], ['จะ', 'ติดต่อ', 'อาจารย์', 'ได้', 'ยัง', 'ไง'], ['จะ', 'ติดต่อ', 'ดอกเตอร์', 'ได้', 'ยัง', 'ไง'], ['จะ', 'ติดต่อ', 'ดร', 'ได้', 'ยัง', 'ไง'], ['อาจารย์', 'อยู่', 'ไหม'], ['อาจารย์', 'อยู่', 'ไหม'], ['อาจารย์', 'อยู่', 'ไหม'], ['อาจารย์', 'อยู่', 'ไหม'], ['คุณ', 'อยู่', 'ไหม'], ['คุณ', 'อยู่', 'ไหม'], ['ดอกเตอร์', 'อยู่', 'ไหม'], ['ดอกเตอร์', 'อยู่', 'ไหม'], ['ดร', 'อยู่', 'ไหม'], ['ดร', 'อยู่', 'ไหม'], ['อาจารย์', 'อยู่', 'ไหน'], ['อาจารย์', 'อยู่', 'ไหน'], ['อาจารย์', 'อยู่', 'ไหน'], ['อาจารย์', 'อยู่', 'ไหน'], ['คุณ', 'อยู่',

In [64]:
from sklearn.metrics import r2_score

def performance_metric(y_true, y_predict):
    """ Calculates and returns the performance score between 
        true and predicted values based on the metric chosen. """
    
    # TODO: Calculate the performance score between 'y_true' and 'y_predict'
    score = r2_score(y_true, y_predict)
    
    # Return the score
    return score

In [65]:
# TODO: Import 'train_test_split'

from sklearn.model_selection import train_test_split

# TODO: Shuffle and split the data into training and testing subsets
X_train, X_test, y_train, y_test = train_test_split(encoded_train_x_datasets, encoded_train_y_datasets, test_size=0.2, random_state=42)

# Success
print("Training and testing split was successful.")

Training and testing split was successful.


In [97]:
from sklearn.metrics import make_scorer
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import ShuffleSplit
import numpy as np

def fit_model(X, y, model):
    """ Performs grid search over the 'max_depth' parameter for a 
        decision tree regressor trained on the input data [X, y]. """
    
    # Create cross-validation sets from the training data
    # sklearn version 0.18: ShuffleSplit(n_splits=10, test_size=0.1, train_size=None, random_state=None)
    # sklearn versiin 0.17: ShuffleSplit(n, n_iter=10, test_size=0.1, train_size=None, random_state=None)
    cv_sets = ShuffleSplit(X.shape[0], n_iter = 10, test_size = 0.20, random_state = 0)


    # TODO: Create a dictionary for the parameter 'max_depth' with a range from 1 to 10
    # params = {'epochs ':np.arange(50,300,50), 'batch_size':np.arange(10,50,5)}
    # params = dict(epochs=np.arange(20,180,25), loss=['categorical_crossentropy', 'mean_squared_error'], optimizer=['adam', 'Nadam', 'Adamax', 'Adadelta', 'RMSprop'])
    params = {'optimizers':['adam', 'Nadam', 'Adamax', 'Adadelta', 'RMSprop'], 
              'epochs':np.arange(20,180,25),
              'losses':['categorical_crossentropy', 'mean_squared_error']}

    # TODO: Create the grid search cv object --> GridSearchCV()
    # Make sure to include the right parameters in the object:
    # (estimator, param_grid, scoring, cv) which have values 'regressor', 'params', 'scoring_fnc', and 'cv_sets' respectively.
    grid = GridSearchCV(estimator=model, param_grid=params, cv=cv_sets)

    # Fit the grid search object to the data to compute the optimal model
    grid = grid.fit(X, y)

    # Return the optimal model after fitting the data
    return grid.best_estimator_

In [95]:
def create_model(losses='categorical_crossentropy', optimizers='adam'):
    max_features = 2000
    model = Sequential()
    model.add(Embedding(max_features, max_word_lenght))
    model.add(LSTM(max_word_lenght, dropout=0.1, recurrent_dropout=0.1))
    model.add(Dense(max_word_lenght, activation='sigmoid'))
    model.add(Dense(number_of_category, activation='softmax')) #softmax used for highlight the largest values and suppress values which are significantly below the maximum value

    model.compile(loss=losses,
                optimizer=optimizers,
                metrics=['accuracy'])
    
    '''model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])'''

    return model

In [88]:
from keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_fn=create_model, verbose=1)

In [101]:
# Fit the training data to the model using grid search
reg = fit_model(X_train, y_train, model)

# Produce the value for 'max_depth'
print("Parameters for the optimal model.",reg)
print(reg.get_params)

Epoch 1/20
129/129 [==============================] - 15s 119ms/step - loss: 1.7956 - acc: 0.1550
Epoch 2/20
129/129 [==============================] - 0s 913us/step - loss: 1.7246 - acc: 0.1550
Epoch 3/20
129/129 [==============================] - 0s 898us/step - loss: 1.6688 - acc: 0.1550
Epoch 4/20
129/129 [==============================] - 0s 822us/step - loss: 1.6274 - acc: 0.1550
Epoch 5/20
129/129 [==============================] - 0s 779us/step - loss: 1.5811 - acc: 0.1550
Epoch 6/20
129/129 [==============================] - 0s 786us/step - loss: 1.5227 - acc: 0.1550
Epoch 7/20
129/129 [==============================] - 0s 992us/step - loss: 1.4587 - acc: 0.1783
Epoch 8/20
129/129 [==============================] - 0s 940us/step - loss: 1.3719 - acc: 0.3178
Epoch 9/20
129/129 [==============================] - 0s 1ms/step - loss: 1.3380 - acc: 0.3178
Epoch 10/20
129/129 [==============================] - 0s 1ms/step - loss: 1.3360 - acc: 0.3178
Epoch 11/20
129/129 [===========

129/129 [==============================] - 0s 777us/step - loss: 1.7861 - acc: 0.3178
Epoch 3/20
129/129 [==============================] - 0s 754us/step - loss: 1.7452 - acc: 0.3178
Epoch 4/20
129/129 [==============================] - 0s 731us/step - loss: 1.6995 - acc: 0.3178
Epoch 5/20
129/129 [==============================] - 0s 733us/step - loss: 1.6517 - acc: 0.3178
Epoch 6/20
129/129 [==============================] - 0s 817us/step - loss: 1.5902 - acc: 0.3178
Epoch 7/20
129/129 [==============================] - 0s 740us/step - loss: 1.5306 - acc: 0.3178
Epoch 8/20
129/129 [==============================] - 0s 566us/step - loss: 1.4769 - acc: 0.3178
Epoch 9/20
129/129 [==============================] - 0s 772us/step - loss: 1.4137 - acc: 0.3178
Epoch 10/20
129/129 [==============================] - 0s 650us/step - loss: 1.3667 - acc: 0.3178
Epoch 11/20
129/129 [==============================] - 0s 830us/step - loss: 1.3197 - acc: 0.3178
Epoch 12/20
129/129 [==================

129/129 [==============================] - 0s 801us/step - loss: 1.5288 - acc: 0.1550
Epoch 4/20
129/129 [==============================] - 0s 917us/step - loss: 1.4911 - acc: 0.1628
Epoch 5/20
129/129 [==============================] - 0s 777us/step - loss: 1.4574 - acc: 0.3411
Epoch 6/20
129/129 [==============================] - 0s 785us/step - loss: 1.4182 - acc: 0.3411
Epoch 7/20
129/129 [==============================] - 0s 777us/step - loss: 1.3896 - acc: 0.3411
Epoch 8/20
129/129 [==============================] - 0s 785us/step - loss: 1.3696 - acc: 0.3411
Epoch 9/20
129/129 [==============================] - 0s 793us/step - loss: 1.3516 - acc: 0.3411
Epoch 10/20
129/129 [==============================] - 0s 816us/step - loss: 1.3428 - acc: 0.3411
Epoch 11/20
129/129 [==============================] - 0s 762us/step - loss: 1.3185 - acc: 0.3411
Epoch 12/20
129/129 [==============================] - 0s 871us/step - loss: 1.3090 - acc: 0.3411
Epoch 13/20
129/129 [=================

129/129 [==============================] - 0s 847us/step - loss: 1.3541 - acc: 0.3333
Epoch 5/20
129/129 [==============================] - 0s 855us/step - loss: 1.3205 - acc: 0.3333
Epoch 6/20
129/129 [==============================] - 0s 847us/step - loss: 1.3030 - acc: 0.3333
Epoch 7/20
129/129 [==============================] - 0s 871us/step - loss: 1.2694 - acc: 0.5814
Epoch 8/20
129/129 [==============================] - 0s 886us/step - loss: 1.2342 - acc: 0.4651
Epoch 9/20
129/129 [==============================] - 0s 832us/step - loss: 1.1873 - acc: 0.6047
Epoch 10/20
129/129 [==============================] - 0s 839us/step - loss: 1.1174 - acc: 0.5271
Epoch 11/20
129/129 [==============================] - 0s 832us/step - loss: 1.0265 - acc: 0.7287
Epoch 12/20
129/129 [==============================] - 0s 832us/step - loss: 1.0007 - acc: 0.7674
Epoch 13/20
129/129 [==============================] - 0s 847us/step - loss: 0.8735 - acc: 0.7597
Epoch 14/20
129/129 [================

129/129 [==============================] - 0s 1ms/step - loss: 1.3107 - acc: 0.3411
Epoch 6/20
129/129 [==============================] - 0s 964us/step - loss: 1.3000 - acc: 0.3721
Epoch 7/20
129/129 [==============================] - 0s 1ms/step - loss: 1.3039 - acc: 0.3488
Epoch 8/20
129/129 [==============================] - 0s 972us/step - loss: 1.3026 - acc: 0.3178
Epoch 9/20
129/129 [==============================] - 0s 999us/step - loss: 1.2963 - acc: 0.3256
Epoch 10/20
129/129 [==============================] - 0s 944us/step - loss: 1.2691 - acc: 0.4109
Epoch 11/20
129/129 [==============================] - 0s 972us/step - loss: 1.2609 - acc: 0.3721
Epoch 12/20
129/129 [==============================] - 0s 1ms/step - loss: 1.2376 - acc: 0.5736
Epoch 13/20
129/129 [==============================] - 0s 975us/step - loss: 1.1963 - acc: 0.4651
Epoch 14/20
129/129 [==============================] - 0s 960us/step - loss: 1.1497 - acc: 0.4496
Epoch 15/20
129/129 [=====================

...

129/129 [==============================] - 0s 2ms/step - loss: 0.2008 - acc: 0.2636
Epoch 3/20
129/129 [==============================] - 0s 2ms/step - loss: 0.1975 - acc: 0.3178
Epoch 4/20
129/129 [==============================] - 0s 2ms/step - loss: 0.1955 - acc: 0.3178
Epoch 5/20
129/129 [==============================] - 0s 2ms/step - loss: 0.1934 - acc: 0.3178
Epoch 6/20
129/129 [==============================] - 0s 2ms/step - loss: 0.1912 - acc: 0.3178
Epoch 7/20
129/129 [==============================] - 0s 2ms/step - loss: 0.1894 - acc: 0.3178
Epoch 8/20
129/129 [==============================] - 0s 1ms/step - loss: 0.1874 - acc: 0.3178
Epoch 9/20
129/129 [==============================] - 0s 2ms/step - loss: 0.1855 - acc: 0.3488
Epoch 10/20
129/129 [==============================] - 0s 1ms/step - loss: 0.1831 - acc: 0.3488
Epoch 11/20
129/129 [==============================] - 0s 2ms/step - loss: 0.1805 - acc: 0.3643
Epoch 12/20
129/129 [==============================] - 0s 2

129/129 [==============================] - 0s 1ms/step - loss: 0.1942 - acc: 0.3101
Epoch 5/20
129/129 [==============================] - 0s 1ms/step - loss: 0.1918 - acc: 0.3101
Epoch 6/20
129/129 [==============================] - 0s 1ms/step - loss: 0.1899 - acc: 0.3101
Epoch 7/20
129/129 [==============================] - 0s 1ms/step - loss: 0.1887 - acc: 0.3101
Epoch 8/20
129/129 [==============================] - 0s 1ms/step - loss: 0.1873 - acc: 0.3101
Epoch 9/20
129/129 [==============================] - 0s 1ms/step - loss: 0.1859 - acc: 0.3101
Epoch 10/20
129/129 [==============================] - 0s 1ms/step - loss: 0.1850 - acc: 0.3101
Epoch 11/20
129/129 [==============================] - 0s 1ms/step - loss: 0.1842 - acc: 0.3101
Epoch 12/20
129/129 [==============================] - 0s 1ms/step - loss: 0.1835 - acc: 0.3101
Epoch 13/20
129/129 [==============================] - 0s 1ms/step - loss: 0.1828 - acc: 0.3101
Epoch 14/20
129/129 [==============================] - 0s

129/129 [==============================] - 0s 2ms/step - loss: 0.1854 - acc: 0.3333
Epoch 8/20
129/129 [==============================] - 0s 2ms/step - loss: 0.1851 - acc: 0.2326
Epoch 9/20
129/129 [==============================] - 0s 2ms/step - loss: 0.1842 - acc: 0.4031
Epoch 10/20
129/129 [==============================] - 0s 2ms/step - loss: 0.1836 - acc: 0.3023
Epoch 11/20
129/129 [==============================] - 0s 2ms/step - loss: 0.1835 - acc: 0.3023
Epoch 12/20
129/129 [==============================] - 0s 2ms/step - loss: 0.1831 - acc: 0.3023
Epoch 13/20
129/129 [==============================] - 0s 2ms/step - loss: 0.1824 - acc: 0.3488
Epoch 14/20
129/129 [==============================] - 0s 2ms/step - loss: 0.1824 - acc: 0.5271
Epoch 15/20
129/129 [==============================] - 0s 2ms/step - loss: 0.1825 - acc: 0.3566
Epoch 16/20
129/129 [==============================] - 0s 2ms/step - loss: 0.1825 - acc: 0.3488
Epoch 17/20
129/129 [==============================] -

129/129 [==============================] - 0s 2ms/step - loss: 0.1787 - acc: 0.3721
Epoch 11/20
129/129 [==============================] - 0s 2ms/step - loss: 0.1786 - acc: 0.3333
Epoch 12/20
129/129 [==============================] - 0s 2ms/step - loss: 0.1787 - acc: 0.3721
Epoch 13/20
129/129 [==============================] - 0s 2ms/step - loss: 0.1787 - acc: 0.3178
Epoch 14/20
129/129 [==============================] - 0s 2ms/step - loss: 0.1787 - acc: 0.3256
Epoch 15/20
129/129 [==============================] - 0s 2ms/step - loss: 0.1781 - acc: 0.3721
Epoch 16/20
129/129 [==============================] - 0s 2ms/step - loss: 0.1782 - acc: 0.3721
Epoch 17/20
129/129 [==============================] - 0s 2ms/step - loss: 0.1782 - acc: 0.3721
Epoch 18/20
129/129 [==============================] - 0s 2ms/step - loss: 0.1783 - acc: 0.3178
Epoch 19/20
129/129 [==============================] - 0s 2ms/step - loss: 0.1785 - acc: 0.3178
Epoch 20/20
129/129 [==============================]

KeyboardInterrupt: 